## Import Library & Basic Setting

In [53]:
import pandas as pd
import os
import nest_asyncio
from dotenv import load_dotenv

from autorag.data.qa.schema import Corpus
from llama_index.llms.openai import OpenAI

from autorag.data.qa.filter.dontknow import (
    dontknow_filter_rule_based,
    dontknow_filter_llama_index
)
from autorag.data.qa.generation_gt.llama_index_gen_gt import (
    make_basic_gen_gt,
    make_concise_gen_gt
)
from autorag.data.qa.query.llama_gen_query import (
    factoid_query_gen,
    concept_completion_query_gen
)
from autorag.data.qa.sample import random_single_hop
from autorag.data.qa.evolve.llama_index_query_evolve import (
    reasoning_evolve_ragas,
    conditional_evolve_ragas,
    compress_ragas
)

In [5]:
nest_asyncio.apply()

In [11]:
load_dotenv()

True

In [3]:
root_dir = os.getcwd()
data_dir = os.path.join(root_dir, "data")

## Get Corpus Data

In [6]:
corpus_data_path = os.path.join(data_dir, "corpus.parquet")
corpus_df = pd.read_parquet(corpus_data_path, engine='pyarrow')

In [7]:
corpus_df.head()

,doc_id,contents,path,start_end_idx,metadata
0,20f100f6-97de-4c53-a668-8af728890084,# 중소벤처기업부 공고 제2024–300호# 2024년 글로벌 팁스(Global T...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
1,799a0806-43a7-40a7-b4b7-d43a989e993c,중소벤처기업부 장관# 1 사업개요- 사업목적 : 미래 신성장 동력 발굴과 글로벌 선...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
2,edf59aa9-38a6-4d98-9525-df8549d94716,# 서류 및 발표평가 결과에 최대 5점까지 가점 부여|가점 세부 항목|점수|비고|\...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
3,08de4eac-4d89-44dc-8d46-e53b23a49fbd,※ 세부 일정은 대내‧외 사정에 의해 변경될 수 있음|공 고|창업기업 신청 ‧ 접수...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."
4,eb64ddf1-dcbf-4e2d-a5b1-827d806645cd,# 6 유의사항◈ 창업지원사업에 신청하는 창업기업은 관련 법령에 따라 다음 사항에 ...,/home/martinus/workspace/RAG-Evaluation/Parsin...,"[0, 0]","{'last_modified_datetime': '2025-04-20', 'next..."


In [8]:
corpus_df.iloc[0]

doc_id                        20f100f6-97de-4c53-a668-8af728890084
contents         # 중소벤처기업부 공고 제2024–300호# 2024년 글로벌 팁스(Global T...
path             /home/martinus/workspace/RAG-Evaluation/Parsin...
start_end_idx                                               [0, 0]
metadata         {'last_modified_datetime': '2025-04-20', 'next...
Name: 0, dtype: object

In [9]:
corpus_df.iloc[0]["contents"]

'# 중소벤처기업부 공고 제2024–300호# 2024년 글로벌 팁스(Global TIPS) 창업기업 모집 공고해외투자와 연계하여 유망 창업기업의 해외 진출 및 글로벌 기업으로의 성장을 지원하는 『2024년 글로벌 팁스(Global TIPS)』에 참여할 창업기업을 다음과 같이 모집합니다.2024년 5월 7일'

## Whole QA Creation Process

In [14]:
llm = OpenAI()
initial_corpus = Corpus(corpus_df=corpus_df)
initial_qa = (
    initial_corpus.sample(random_single_hop, n=3) # sample three
    .map(
        lambda df: df.reset_index(drop=True)
    )
    .make_retrieval_gt_contents()
    .batch_apply(
        factoid_query_gen, # query generation
        llm=llm,
        lang='ko'
    )
    .batch_apply(
        make_basic_gen_gt, # answer generation (basic)
        llm=llm,
        lang='ko'
    )
    .batch_apply(
        make_concise_gen_gt, # answer generation (concise)
        llm=llm,
        lang='ko'
    )
    .filter(
        dontknow_filter_rule_based, # filter don't know
        lang='ko'
    )
)

[05/16/25 14:07:53] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=920673;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=114461;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=655017;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=824535;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=867796;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=932612;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:07:54] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=995142;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=499993;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:07:55] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=537357;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=289890;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:07:56] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=292768;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=299827;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:07:57] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=155893;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=894498;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=926544;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=125049;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=877462;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=786249;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [15]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,0a6688e9-6695-4607-a2ea-7ce34a897a25,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...,창업기업이 신청자격을 갖추기 위해 필요한 조건은 무엇입니까?,[창업기업이 신청자격을 갖추기 위해 필요한 조건은 다음과 같습니다:\n1. 중소기업...
1,1461bf8d-f20a-4d2d-987c-ad71bb5f912f,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...,"해외VC 추천서에서 가점 관련 증빙서류를 언제 제출해야 하며, 미제출 시 어떤 일이...","[가점 관련 증빙서류는 사업신청 시 제출해야 하며, 미제출 시 가점이 불인정됩니다...."
2,a63b5c1f-2251-4b0c-9a2d-ce105549d780,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...,창업기업 선정평가의 평가 절차는 무엇으로 구성되어 있나요?,[창업기업 선정평가의 평가 절차는 다음과 같이 구성되어 있습니다:\n1. 요건검토\...


## Sample Retrieval GT (Randomly)

In [24]:
initial_corpus = Corpus(corpus_df=corpus_df)
initial_qa = initial_corpus.sample(random_single_hop, n=3)

In [25]:
initial_qa.data

,qid,retrieval_gt
0,a111c1a3-2ccc-4268-bffa-2371a8dbb133,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]]
1,3eb9bc6b-7113-4f4f-a0e9-0abaf77b52ae,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]]
2,c9cac7fa-cc51-41a9-bc12-6456070ae6f1,[[cf502805-4bec-416d-bdeb-bd52780c02cc]]


In [26]:
# Corpus dataset에서 각 retrieval_gt id에 해당하는 데이터를 가져옴
initial_qa = (initial_qa.map(
    lambda df: df.reset_index(drop=True)
)
.make_retrieval_gt_contents())
# initial_qa = initial_qa.make_retrieval_gt_contents()

In [27]:
initial_qa.data

,qid,retrieval_gt,retrieval_gt_contents
0,a111c1a3-2ccc-4268-bffa-2371a8dbb133,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...
1,3eb9bc6b-7113-4f4f-a0e9-0abaf77b52ae,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...
2,c9cac7fa-cc51-41a9-bc12-6456070ae6f1,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...


## Query Generation

In [28]:
factoid_qa = initial_qa.batch_apply(
    factoid_query_gen,
    llm=llm,
    lang='ko'
)

[05/16/25 14:19:18] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=254540;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=52859;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=930758;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=598559;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=510419;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=664053;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [31]:
factoid_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query
0,a111c1a3-2ccc-4268-bffa-2371a8dbb133,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...,창업기업이 신청자격을 갖추기 위해 필요한 조건은 무엇입니까?
1,3eb9bc6b-7113-4f4f-a0e9-0abaf77b52ae,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...,창업기업 여부를 확인하기 위해 요청될 수 있는 추가적인 서류는 무엇입니까?
2,c9cac7fa-cc51-41a9-bc12-6456070ae6f1,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...,창업기업 선정평가의 평가 절차는 무엇으로 구성되어 있나요?


In [32]:
factoid_qa.data['query']

0            창업기업이 신청자격을 갖추기 위해 필요한 조건은 무엇입니까?
1    창업기업 여부를 확인하기 위해 요청될 수 있는 추가적인 서류는 무엇입니까?
2             창업기업 선정평가의 평가 절차는 무엇으로 구성되어 있나요?
Name: query, dtype: object

In [34]:
concept_completion_qa = initial_qa.batch_apply(
    concept_completion_query_gen,
    llm=llm,
    lang='ko'
)

[05/16/25 14:21:24] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=347830;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=943840;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=932920;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=304064;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:21:25] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=445829;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=253014;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [35]:
concept_completion_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query
0,a111c1a3-2ccc-4268-bffa-2371a8dbb133,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...,어떤 조건을 충족하는 창업기업이 중소기업창업 지원을 받을 수 있나요?
1,3eb9bc6b-7113-4f4f-a0e9-0abaf77b52ae,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...,사업계획서 양식을 활용해야 하는 이유는 무엇인가요?
2,c9cac7fa-cc51-41a9-bc12-6456070ae6f1,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...,창업기업 선정을 위한 평가 절차는 어떻게 되나요?


In [36]:
concept_completion_qa.data['query']

0    어떤 조건을 충족하는 창업기업이 중소기업창업 지원을 받을 수 있나요?
1              사업계획서 양식을 활용해야 하는 이유는 무엇인가요?
2               창업기업 선정을 위한 평가 절차는 어떻게 되나요?
Name: query, dtype: object

## Query Evolving

In [38]:
# Reasoning Evolving

reasoning_qa = factoid_qa.batch_apply(
    reasoning_evolve_ragas,
    llm=llm,
    lang='ko'
)

[05/16/25 14:24:32] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=34865;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=194133;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=331229;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=95247;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=777274;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=947865;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [ ]:
reasoning_qa.data['query']

0          창업기업이 신청자격을 갖추기 위해 필요한 조건은 무엇입니까?
1    창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 무엇인가요?
2            창업기업 선정평가의 평가 절차는 어떻게 구성되어 있나요?
Name: query, dtype: object

In [41]:
# Conditional Evolving

conditional_qa = factoid_qa.batch_apply(
    conditional_evolve_ragas,
    llm=llm,
    lang='ko'
)

[05/16/25 14:26:31] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=941893;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=324370;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=20549;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=321295;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=541305;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=169445;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [42]:
conditional_qa.data['query']

0                     창업기업이 신청자격을 얻기 위해 필요한 조건은 무엇입니까?
1              창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 무엇입니까?
2    창업기업 선정평가의 평가 절차는 어떻게 구성되어 있습니까? 평가는 몇 단계로 이루어...
Name: query, dtype: object

In [44]:
compress_qa = factoid_qa.batch_apply(
    compress_ragas,
    llm=llm,
    lang='ko'
)

[05/16/25 14:27:24] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=327447;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=581923;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=198051;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=137607;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=761103;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=758839;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [45]:
compress_qa.data['query']

0          창업기업 자격을 얻기 위한 필수 조건은 무엇인가요?
1    창업기업 자격 확인을 위해 요청될 수 있는 서류는 무엇인가요?
2            창업기업 선정평가의 평가 절차는 어떻게 되나요?
Name: query, dtype: object

## Answer Generation

In [46]:
qa_answer = conditional_qa.batch_apply(
    make_basic_gen_gt,
    llm=llm,
    lang='ko'
)

[05/16/25 14:29:56] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=114201;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=579277;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:29:57] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=436662;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=93099;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=131248;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=598894;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [47]:
qa_answer.data[['query', 'generation_gt']]

,query,generation_gt
0,창업기업이 신청자격을 얻기 위해 필요한 조건은 무엇입니까?,[창업기업이 신청자격을 얻기 위해 필요한 조건은 다음과 같습니다:\n1. 중소기업의...
1,창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 무엇입니까?,[창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 '창업기업 확인서'입니...
2,창업기업 선정평가의 평가 절차는 어떻게 구성되어 있습니까? 평가는 몇 단계로 이루어...,"[창업기업 선정평가는 총 2단계로 이루어져 있습니다. 첫 번째는 서류평가이고, 이때..."


In [48]:
qa_answer = qa_answer.batch_apply(
    make_concise_gen_gt,
    llm=llm,
    lang='ko'
)

[05/16/25 14:31:02] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=20745;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=459763;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:31:03] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=944399;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=895979;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=458760;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=548528;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [49]:
qa_answer.data[['query', 'generation_gt']]

,query,generation_gt
0,창업기업이 신청자격을 얻기 위해 필요한 조건은 무엇입니까?,[창업기업이 신청자격을 얻기 위해 필요한 조건은 다음과 같습니다:\n1. 중소기업의...
1,창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 무엇입니까?,[창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 '창업기업 확인서'입니...
2,창업기업 선정평가의 평가 절차는 어떻게 구성되어 있습니까? 평가는 몇 단계로 이루어...,"[창업기업 선정평가는 총 2단계로 이루어져 있습니다. 첫 번째는 서류평가이고, 이때..."


In [50]:
qa_answer.data.iloc[2]['generation_gt']

['창업기업 선정평가는 총 2단계로 이루어져 있습니다. 첫 번째는 서류평가이고, 이때는 제출한 사업계획서를 토대로 서면 평가가 이루어집니다. 두 번째는 발표평가로, 창업기업 대표자가 발표를 하고 질의응답을 받는 과정을 거칩니다. 최종 선정은 발표평가를 통해 이루어지며, 정부지원사업비가 확정됩니다.',
 '창업기업 선정평가는 총 2단계로 구성되어 있습니다. 첫 번째는 서류평가이며, 두 번째는 발표평가입니다. 서류평가에서는 제출한 사업계획서를 기반으로 서면 평가가 이루어지고, 발표평가에서는 창업기업 대표자가 발표하고 질의응답을 받습니다.']

## Filtering

In [51]:
final_qa = qa_answer.filter(
    dontknow_filter_rule_based,
    lang='ko'
).map(
    lambda df: df.reset_index(drop=True)
)

In [52]:
final_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,a111c1a3-2ccc-4268-bffa-2371a8dbb133,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...,창업기업이 신청자격을 얻기 위해 필요한 조건은 무엇입니까?,[창업기업이 신청자격을 얻기 위해 필요한 조건은 다음과 같습니다:\n1. 중소기업의...
1,3eb9bc6b-7113-4f4f-a0e9-0abaf77b52ae,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...,창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 무엇입니까?,[창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 '창업기업 확인서'입니...
2,c9cac7fa-cc51-41a9-bc12-6456070ae6f1,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...,창업기업 선정평가의 평가 절차는 어떻게 구성되어 있습니까? 평가는 몇 단계로 이루어...,"[창업기업 선정평가는 총 2단계로 이루어져 있습니다. 첫 번째는 서류평가이고, 이때..."


In [54]:
final_qa = qa_answer.batch_filter(
    dontknow_filter_llama_index,
    llm=llm,
    lang='ko'
).map(
    lambda df: df.reset_index(drop=True)
)

[05/16/25 14:36:02] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=613866;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=902577;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=107890;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=799669;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=521340;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=8594;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

[05/16/25 14:36:03] INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=726498;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=760322;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=374437;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=608300;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

                    INFO     [_client.py:1786] >> HTTP Request: POST                                ]8;id=143319;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=280806;file:///opt/anaconda3/envs/llm/lib/python3.10/site-packages/httpx/_client.py#1786\1786]8;;\
                             https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"                          

In [55]:
final_qa.data

,qid,retrieval_gt,retrieval_gt_contents,query,generation_gt
0,a111c1a3-2ccc-4268-bffa-2371a8dbb133,[[a8504eac-0f1d-48cf-b59e-6b22cb2a9e1f]],[[2 신청자격 및 요건\n□ 신청자격\n◦ ｢중소기업기본법｣ 제2조제1항에 따른 ...,창업기업이 신청자격을 얻기 위해 필요한 조건은 무엇입니까?,[창업기업이 신청자격을 얻기 위해 필요한 조건은 다음과 같습니다:\n1. 중소기업의...
1,3eb9bc6b-7113-4f4f-a0e9-0abaf77b52ae,[[927ad9c5-b55c-4f7c-b2ad-df9c06b27577]],[[** 해외VC 추천서 : “[별첨2] 증빙서류 제출목록” 참조*** 반드시 동 ...,창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 무엇입니까?,[창업기업 여부를 확인하기 위해 요청될 수 있는 추가 서류는 '창업기업 확인서'입니...
2,c9cac7fa-cc51-41a9-bc12-6456070ae6f1,[[cf502805-4bec-416d-bdeb-bd52780c02cc]],[[# 5 평가 및 선정□ 평가 절차 : 총 2단계 평가 (서류 + 발표(현장실사)...,창업기업 선정평가의 평가 절차는 어떻게 구성되어 있습니까? 평가는 몇 단계로 이루어...,"[창업기업 선정평가는 총 2단계로 이루어져 있습니다. 첫 번째는 서류평가이고, 이때..."
